## 4. 预处理方法

本notebook对应[这一篇blog post](https://hanqiu92.github.io/blogs/2020/LP_dual_simplex_solver_4_202004/)中的内容，主要包括一些基本预处理手段的实现。

首先仍然是一些准备工作：导入相关的计算工具包、从文件util_lec_4.py和util.py中读取前几个notebook中完成的类。

In [1]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse import csc_matrix,csr_matrix,coo_matrix
from util import *
from util_lec_4 import *

### 问题形式转换

由于问题形式转换在DS流程中的必要性，在此前DualSimplexSolver.solve函数里已经实现过；下面的代码对相关逻辑进行了封装。

In [2]:
def transform(A,b,sense,c,l,u):
    '''
    将（单侧）不等式约束转换为等式约束
    sense参数指定了约束方向
    '''
    n,m = A.shape
    c = np.concatenate([c,np.zeros((n,))])
    l = np.concatenate([l,np.zeros((n,))])
    u = np.concatenate([u,np.zeros((n,))])
    for colidx in range(n):
        ## 对非等式对应的逻辑变量加上上下界
        if sense[colidx] == 1: ## G
            l[m+colidx] = -INF
        elif sense[colidx] == -1: ## L
            u[m+colidx] = INF
    A = sp.hstack([A,sp.eye(n)],format='csc')
    return A,b,c,l,u

需要注意的是，上面的函数实现只考虑了单侧约束；这是因为我们在读取问题阶段（util.read_mps）直接对双侧约束进行了形式转换。根据blog post中的讨论，将问题形式转换前置可能会影响预处理手段的作用空间，因此这里的做法并不是最高效的。

### 系数缩放

在这一小节中，我将实现多种行/列权重计算和调整方式（$l_2,l_{\infty},$geomean），并通过组合这些调整方式实现一个系数缩放流程。同时，我也实现了后处理步骤中对解重缩放的流程。

由于输入的矩阵是稀疏存储的，为了提高计算效率，有必要对计算方法进行设计。在下面的实现中，我选用了COO格式来存储稀疏矩阵$A$，这样可以对每个矩阵中的非零元素灵活地提取相应的行和列的权重并进行计算。

In [3]:
def scaling(A,b,c,l,u):
    '''
    对问题系数进行缩放，以降低条件数和数值难度
    '''

    def scale_row(A,w_row):
        '''
        根据行权重，对稀疏矩阵A的系数进行调整
        假设A是以COO格式存储的
        '''
        w_row[w_row == 0] = 1 ## 如果有权重为0，则重置为1
        A.data /= w_row[A.row] ## 对每个系数，除以相应的行权重
        return A,w_row

    def scale_col(A,w_col):
        '''
        根据列权重，对稀疏矩阵A的系数进行调整
        假设A是以COO格式存储的
        '''
        w_col[w_col == 0] = 1 ## 如果有权重为0，则重置为1
        A.data /= w_col[A.col] ## 对每个系数，除以相应的列权重
        return A,w_col

    def scale(A,w_row,w_col):
        '''
        根据行和列的权重，对稀疏矩阵A的系数进行调整
        假设A是以COO格式存储的
        '''
        A,w_row = scale_row(A,w_row) ## 根据行权重进行调整
        A,w_col = scale_col(A,w_col) ## 根据列权重进行调整
        return A,w_row,w_col

    def l2_scale(A):
        '''
        根据l2范数进行系数调整
        '''
        ## 行和列的权重为l2范数
        A_square = A.multiply(A)
        w_row = np.sqrt(A_square.sum(axis=1).A1)
        w_col = np.sqrt(A_square.sum(axis=0).A1)
        ## 根据行和列的权重，对A的系数进行调整
        A,w_row,w_col = scale(A,np.sqrt(w_row),np.sqrt(w_col))
        return A,w_row,w_col

    def max_scale(A):
        '''
        根据l_{\infty}范数进行系数调整
        '''
        w_row = abs(A).max(axis=1).A[:,0] ## 行权重为每行系数的最大值
        A,w_row = scale_row(A,w_row) ## 根据行权重进行调整
        w_col = abs(A).max(axis=0).A[0,:] ## 列权重为每列系数的最大值
        A,w_col = scale_col(A,w_col) ## 根据列权重进行调整
        return A,w_row,w_col

    def geomean_scale(A):
        '''
        根据几何平均值进行系数调整
        '''
        ## 根据行权重进行调整
        A_abs = abs(A)
        w_row_max = A_abs.max(axis=1).A[:,0] ## 先算出每行系数的最大值
        A_abs.data = 1 / A_abs.data
        w_row_min = A_abs.max(axis=1).A[:,0] ## 再算出每行系数最小值的倒数
        w_row_min[w_row_min == 0] = 1
        w_row = np.sqrt(w_row_max / w_row_min) ## 计算（最大值×最小值）的开方
        A,w_row = scale_row(A,w_row)

        ## 根据列权重进行调整
        A_abs = abs(A)
        w_col_max = A_abs.max(axis=0).A[0,:] ## 先算出每列系数的最大值
        A_abs.data = 1 / A_abs.data
        w_col_min = A_abs.max(axis=0).A[0,:] ## 再算出每列系数最小值的倒数
        w_col_min[w_col_min == 0] = 1
        w_col = np.sqrt(w_col_max / w_col_min)
        A,w_col = scale_col(A,w_col)
        return A,w_row,w_col

    ## scaling的主流程：组合使用多种调整手段
    n,m = A.shape
    A = A.tocoo()
    w_row,w_col = np.ones((n,)),np.ones((m,)) ## 用来保存总体的权重
    A,w_row_tmp,w_col_tmp = l2_scale(A) ## 先根据l2范数进行调整
    w_row,w_col = w_row * w_row_tmp,w_col * w_col_tmp ## 更新总体权重
    A,w_row_tmp,w_col_tmp = max_scale(A) ## 再根据l_{\infty}范数进行调整
    w_row,w_col = w_row * w_row_tmp,w_col * w_col_tmp ## 更新总体权重
    A = A.tocsc()

    ## 在结束对A的更新后，根据总体的行和列权重，对输入向量b,c,l,u的取值进行调整
    ## 注意这里的权重是blog中权重的倒数
    b,c = b / w_row,c / w_col
    l[l > -INF] = l[l > -INF] * w_col[l > -INF]
    u[u < INF] = u[u < INF] * w_col[u < INF]

    return A,b,c,l,u,(w_row,w_col)

def descaling(sol,w_row,w_col):
    '''
    根据预处理时的权重对解进行重缩放处理
    '''
    sol.x /= w_col
    sol.s /= w_col
    sol.lam /= w_row
    return sol

### 问题简化

这一小节将实现blog post中提到的一系列简化手段。在本notebook中，我的主要目标是说明预处理的计算效率，因此为了节省空间对代码做了较多简化，移除了很多功能（包括可行性状态的管理、对偶变量和基的后处理）。

首先是对一些基础的矩阵操作进行封装；后续不同的简化手段都会用到这些操作。这里同样假定存储稀疏矩阵$A$是通过COO格式进行存储的；这种格式允许我们灵活地对行和列进行删除操作。

In [4]:
def mat_reduce(A_coo,bool_data_keep,if_inplace=False):
    '''
    对稀疏矩阵A的元素进行删减
    
    输入参数：
    A_coo 以COO格式存储的矩阵
    bool_data_keep 每个元素是否被保留的布尔值
    if_inplace 是否在原矩阵对象中进行操作
    '''
    data,row,col = A_coo.data[bool_data_keep],A_coo.row[bool_data_keep],A_coo.col[bool_data_keep]
    if if_inplace:
        A_coo.data,A_coo.row,A_coo.col = data,row,col
        return A_coo
    else:
        return coo_matrix((data,(row,col)),shape=A_coo.shape)

def rhs_reduce(A_coo,b,x,bool_col_remove):
    '''
    根据要去除的列的集合J，更新右侧项b \gets b - A_J x_J
    
    输入参数：
    A_coo 以COO格式存储的矩阵
    b 右侧项
    x 原始变量
    bool_col_remove 每个列是否需要被去除的布尔值
    '''
    ## 获取矩阵A中需要被去除的元素及相关信息（行i，列j，值A_ij）
    bool_data_remove = bool_col_remove[A_coo.col]
    row,col,data = A_coo.row[bool_data_remove],A_coo.col[bool_data_remove],A_coo.data[bool_data_remove]
    ## 使用bincount函数，将各A_ij x_j的值聚合到行i上
    b -= np.bincount(row,weights=data*x[col],minlength=A_coo.shape[0])
    return b

接下来，我将通过类Reducer对各简化手段进行统一管理，以在简化过程（reduce方法）中能够综合利用各种简化手段，并在后处理过程（restore方法）中能够正确地根据简化中间结果对解进行恢复。

一些实现上值得关注的点包括：
* 需要引入变量对简化过程的中间结果进行保存，用于后续的后处理过程。由于这里只考虑了原始变量的后处理，因此只维护了固定的原始变量值x_preprocess和相关关系A_preprocess。
* 由于在简化过程中会多次调用不同的简化手段，而简化的效果是不确定的（去除大多数行和列是小概率事件），因此使用一组固定长度的布尔型向量rows_keep, cols_keep来对还保留着的行和列信息进行管理。（如果用list维护，从内存上优化空间不大，但在list下标与行/列的位置的映射管理上需要进行更多计算。）同时，在调用各简化手段的过程中不直接对矩阵的shape进行调整，而是先消除元素，在简化过程的最后再统一改变shape。
* 由于Python的循环计算效率低，在下面的稀疏矩阵计算中为了提高效率引入了一些不太好理解的trick。如果用C++等语言实现就不需要这么复杂。

In [5]:
class Reducer(object):
    def __init__(self,A,b,sense,c,l,u):
        ## 输入初始化
        self.A = A.copy()
        self.A_coo = A.tocoo()
        self.b,self.c = b.copy(),c.copy()
        self.sense,self.l,self.u = sense.copy(),l.copy(),u.copy()
        self.n,self.m = A.shape

        ## 中间变量初始化
        self.x_preprocess = self.l.copy()
        self.x_preprocess[self.l <= -INF] = 0
        self.A_preprocess = []
        self.rows_to_keep = np.ones((self.n,),dtype=bool)
        self.cols_to_keep = np.ones((self.m,),dtype=bool)
        
    def check_empty_row(self):
        '''
        检查空行
        '''
        row_nnz_size = self.A_coo.getnnz(axis=1) ## 获取每个行非零元素的数量
        bool_empty_row = (row_nnz_size == 0) & self.rows_to_keep ## 获取未去除行中的空行
        if np.any(bool_empty_row):
            bool_err = (self.b * self.sense > 0) & bool_empty_row ## 检查可行性
            if np.any(bool_err):
                print('Primal Inf - empty row with inf rhs.')
                exit(1)
                
            self.rows_to_keep = self.rows_to_keep & (~bool_empty_row) ## 保留非空行

    def check_empty_col(self):
        '''
        检查空列
        '''
        col_nnz_size = self.A_coo.getnnz(axis=0) ## 获取每个列非零元素的数量
        bool_empty_col = (col_nnz_size == 0) & self.cols_to_keep ## 获取未去除列中的空列
        if np.any(bool_empty_col):
            bool_c_ge_0,bool_c_le_0 = self.c[bool_empty_col] > 0,self.c[bool_empty_col] < 0
            l_tmp,u_tmp = self.l[bool_empty_col],self.u[bool_empty_col]
            ## 检查对偶可行性
            if np.any( ((bool_c_ge_0) & (l_tmp <= -INF)) | \
                       ((bool_c_le_0) & (u_tmp >= INF))):
                print('Dual Inf - empty col with inf obj.')
                exit(1)
                
            ## 根据目标系数c和上下界确定去除列的x的取值，并保存到x_preprocess中
            x_tmp = l_tmp.copy()
            x_tmp[bool_c_le_0] = u_tmp[bool_c_le_0]
            self.x_preprocess[bool_empty_col] = x_tmp
            self.cols_to_keep = self.cols_to_keep & (~bool_empty_col) ## 保留非空列

    def check_fixed_col(self):
        '''
        检查固定值变量
        '''
        bool_fixed_col = (self.u <= self.l) & self.cols_to_keep ## 用上下界进行判断
        if np.any(bool_fixed_col):
            ## 检查可行性
            if np.any(self.u[bool_fixed_col] < self.l[bool_fixed_col]):
                print('Primal Inf - col with inf bnds.')
                exit(1)
            
            self.x_preprocess[bool_fixed_col] = self.l[bool_fixed_col] ## 将固定值保存到x_preprocess中
            rhs_reduce(self.A_coo,self.b,self.x_preprocess,bool_fixed_col) ## 更新右侧项
            self.cols_to_keep = self.cols_to_keep & (~bool_fixed_col) ## 保留非固定值列
            self.A_coo = mat_reduce(self.A_coo,self.cols_to_keep[self.A_coo.col],if_inplace=True) ## 删减矩阵元素
            
    def check_singleton_row(self):
        '''
        检查单变量约束
        '''
        row_nnz_size = self.A_coo.getnnz(axis=1) ## 获取每个行非零元素的数量
        bool_singleton_row = (row_nnz_size == 1) & self.rows_to_keep ## 获取未去除行中的单元素行
        if np.any(bool_singleton_row):
            ## 获取单元素行的元素及相关信息（行i，列j，值A_ij）
            bool_data = bool_singleton_row[self.A_coo.row] 
            row,col,data = self.A_coo.row[bool_data],self.A_coo.col[bool_data],self.A_coo.data[bool_data]
            sense = self.sense[row] ## 约束方向
            ## 遍历每个行
            for row_,col_,data_,sense_ in zip(row,col,data,sense):
                if ((sense_ >= 0) & (data_ > 0)) | ((sense_ <= 0) & (data_ < 0)):
                    ## 转换为下界约束
                    self.l[col_] = max(self.l[col_],self.b[row_] / data_)
                if ((sense_ <= 0) & (data_ > 0)) | ((sense_ >= 0) & (data_ < 0)):
                    ## 转换为上界约束
                    self.u[col_] = min(self.u[col_],self.b[row_] / data_)
                
            self.rows_to_keep = self.rows_to_keep & (~bool_singleton_row) ## 保留非单变量行
            self.A_coo = mat_reduce(self.A_coo,self.rows_to_keep[self.A_coo.row],if_inplace=True) ## 删减矩阵元素

    def check_obj_col(self):
        '''
        检查最优性隐含变量
        '''
        A_coo = self.A_coo
        ## 计算A_ij * c_j * sense_i的符号，以检查矩阵A中每个元素A_ij和约束方向是否约束变量x_j朝c_j方向前进
        A_tmp = A_coo.copy()
        A_tmp.data = (A_tmp.data * self.sense[A_tmp.row] * self.c[A_tmp.col] >= 0)
        
        bool_no_con_col = np.asarray(A_tmp.sum(axis=0))[0,:] == 0 ## 根据矩阵元素的结果，判断每个列j是否会被约束
        bool_obj_col = bool_no_con_col & self.cols_to_keep ## 获取未去除列中的无约束列
        if np.any(bool_obj_col):
            ## 确定x的值，并将值保存到x_preprocess中
            self.x_preprocess[bool_obj_col] = self.u[bool_obj_col] * (self.c[bool_obj_col] < 0) + \
                                                self.l[bool_obj_col] * (self.c[bool_obj_col] >= 0)
            if np.any(np.abs(self.x_preprocess[bool_obj_col]) >= INF):
                print('Dual Inf - col with inf obj.')
                exit(1)
            rhs_reduce(self.A_coo,self.b,self.x_preprocess,bool_obj_col) ## 更新右侧项
            self.cols_to_keep = self.cols_to_keep & (~bool_obj_col) ## 保留有约束列
            self.A_coo = mat_reduce(self.A_coo,self.cols_to_keep[self.A_coo.col],if_inplace=True) ## 删减矩阵元素

    def check_implied_row(self):
        '''
        检查可行性隐含约束
        '''
        ## 计算每个约束i的隐含上下界b_nega,b_posi
        A_tmp = self.A_coo.tocsr()
        A_posi,A_nega = A_tmp.multiply(A_tmp > 0),A_tmp.multiply(A_tmp < 0)
        b_nega = A_posi.dot(self.l) + A_nega.dot(self.u)
        b_posi = A_posi.dot(self.u) + A_nega.dot(self.l)
        ## 根据b_nega,b_posi和b的相对大小进行判断
        delta_posi,delta_nega = b_posi - self.b,self.b - b_nega
        bool_nega_g,bool_posi_l = delta_nega < -PRIMAL_TOL,delta_posi < -PRIMAL_TOL ## 隐含下(上)界显著大(小)于b
        bool_nega_ge,bool_posi_le = delta_nega <= 0,delta_posi <= 0 ## 隐含下(上)界大(小)于等于b
        bool_nega_eq,bool_posi_eq = delta_nega == 0,delta_posi == 0 ## 隐含下(上)界等于b
        bool_sense_le,bool_sense_ge = self.sense <= 0,self.sense >= 0 ## 约束方向
        ## 检查约束的可行性
        bool_primal_inf = ((bool_sense_le & bool_nega_g) | (bool_sense_ge & bool_posi_l)) & self.rows_to_keep
        if np.any(bool_primal_inf):
            print('Primal Inf - row bnds inf.')
            exit(1)
            
        ## 获取可去除的行以及可固定值的行
        ## 约束 <= 隐含上界 <= b 或者 约束 >= 隐含上界 >= b, 因此sense多余
        bool_redundant_row = ((bool_sense_le & bool_posi_le) | (bool_sense_ge & bool_nega_ge)) & self.rows_to_keep
        bool_fixed_lower_row = bool_sense_le & bool_nega_eq & self.rows_to_keep ## 约束 >= 隐含下界 = b >= 约束
        bool_fixed_upper_row = bool_sense_ge & bool_posi_eq & self.rows_to_keep ## 约束 <= 隐含上界 = b <= 约束
        
        ## 确定所需固定值的变量及固定值
        mat_fixed_lower_row = A_tmp[bool_fixed_lower_row].tocoo()
        mat_fixed_upper_row = A_tmp[bool_fixed_upper_row].tocoo()
        fixed_lower_col = set(mat_fixed_lower_row.col[mat_fixed_lower_row.data >= 0]) | \
                            set(mat_fixed_upper_row.col[mat_fixed_upper_row.data <= 0])
        fixed_upper_col = set(mat_fixed_lower_row.col[mat_fixed_lower_row.data < 0]) | \
                            set(mat_fixed_upper_row.col[mat_fixed_upper_row.data > 0])
        ## 如果有变量同时要固定为上下值，检查可行性
        if len(fixed_lower_col & fixed_upper_col) > 0:
            idx_primal_inf = list(fixed_lower_col & fixed_upper_col)
            if np.sum(self.u[idx_primal_inf] - self.l[idx_primal_inf]) > 0:
                print('Primal Inf - var fixed at both bnds.')

        ## 将固定值存储到x_preprocess中
        fixed_col = list(fixed_lower_col | fixed_upper_col)
        fixed_lower_col,fixed_upper_col = list(fixed_lower_col),list(fixed_upper_col)
        self.x_preprocess[fixed_lower_col] = self.l[fixed_lower_col]
        self.x_preprocess[fixed_upper_col] = self.u[fixed_upper_col]
        ## 更新右侧项
        bool_fixed_col = np.zeros((self.m,),dtype=bool)
        bool_fixed_col[fixed_col] = True
        rhs_reduce(self.A_coo,self.b,self.x_preprocess,bool_fixed_col)
        ## 删除多余的行和列
        self.rows_to_keep = self.rows_to_keep & (~bool_redundant_row)
        self.cols_to_keep[fixed_col] = False
        self.A_coo = mat_reduce(self.A_coo,self.rows_to_keep[self.A_coo.row] & self.cols_to_keep[self.A_coo.col],
                                if_inplace=True) ## 删减矩阵元素

    def check_doubleton_row(self):
        '''
        检查双变量等式约束
        '''
        row_nnz_size = self.A_coo.getnnz(axis=1)  ## 获取每个行非零元素的数量
        bool_doubleton_row = (row_nnz_size == 2) & (self.sense == 0) & self.rows_to_keep ## 获取未去除行中的双元素等式行
        if np.any(bool_doubleton_row):
            ## 遍历每个双变量等式约束
            for row in np.where(bool_doubleton_row)[0]:
                ## 获取该行相关信息
                elem = self.A_coo.row == row
                col = self.A_coo.col[elem]
                if len(col) == 2: ## 确认的确是双变量行
                    ai,bi = self.A_coo.data[elem],self.b[row]
                    ## 根据ai的相对大小进行交换
                    if_first = abs(ai[0]) >= abs(ai[1])
                    col_rm,col_keep = col if if_first else (col[1],col[0])
                    ai_col_rm,ai_col_keep = ai if if_first else (ai[1],ai[0])
                    a_rate,b_rate = ai_col_keep/ai_col_rm,bi/ai_col_rm
                    
                    ## 获取两个列的信息
                    bool_col_rm_data,bool_col_keep_data = self.A_coo.col == col_rm,self.A_coo.col == col_keep
                    col_rm_data,col_rm_row = self.A_coo.data[bool_col_rm_data],self.A_coo.row[bool_col_rm_data]
                    col_keep_data,col_keep_row = self.A_coo.data[bool_col_keep_data],self.A_coo.row[bool_col_keep_data]

                    ## 由于矩阵是稀疏的，通过dict来进行相减以提高效率
                    col_keep_data_by_row = dict()
                    for data_,row_ in zip(col_keep_data,col_keep_row):
                        col_keep_data_by_row[row_] = col_keep_data_by_row.get(row_,0) + data_
                    for data_,row_ in zip(col_rm_data,col_rm_row):
                        col_keep_data_by_row[row_] = col_keep_data_by_row.get(row_,0) - a_rate * data_
                    col_keep_data,col_keep_row = [],[]
                    for row_,data_ in col_keep_data_by_row.items():
                        if abs(data_) > REMOVE_TOL:
                            col_keep_data += [data_]
                            col_keep_row += [row_]
                    col_keep_data = np.array(col_keep_data)
                    col_keep_row = np.array(col_keep_row,dtype=int)
                    col_keep_col = col_keep * np.ones((len(col_keep_row),),dtype=int)

                    ## 根据相减结果更新矩阵
                    bool_data_keep = ~ (bool_col_rm_data | bool_col_keep_data)
                    self.A_coo.data = np.concatenate([self.A_coo.data[bool_data_keep],col_keep_data])
                    self.A_coo.row = np.concatenate([self.A_coo.row[bool_data_keep],col_keep_row])
                    self.A_coo.col = np.concatenate([self.A_coo.col[bool_data_keep],col_keep_col])

                    ## 更新其他输入向量
                    self.b[col_rm_row] -= b_rate * col_rm_data
                    new_bounds = ((bi - ai_col_rm*self.l[col_rm])/ai_col_keep,
                                  (bi - ai_col_rm*self.u[col_rm])/ai_col_keep)
                    new_bounds = new_bounds if new_bounds[1] >= new_bounds[0] else (new_bounds[1],new_bounds[0])
                    self.l[col_keep] = max(self.l[col_keep],new_bounds[0])
                    self.u[col_keep] = min(self.u[col_keep],new_bounds[1])
                    self.c[col_keep] -= a_rate * self.c[col_rm]

                    ## 保留预处理信息，用于后处理
                    self.A_preprocess += [[col_rm,col_keep,ai_col_rm,ai_col_keep,bi]]
                    
                    ## 更新保留行列的信息
                    self.cols_to_keep[col_rm] = False
                    self.rows_to_keep[row] = False
                    
    def drop_row_col(self):
        '''
        根据rows_to_keep和cols_to_keep对各输入矩阵/向量进行化简
        '''
        self.A = self.A_coo.tocsc()
        A = self.A[self.rows_to_keep,:][:,self.cols_to_keep]
        b = self.b[self.rows_to_keep]
        sense = self.sense[self.rows_to_keep]
        c = self.c[self.cols_to_keep]
        l = self.l[self.cols_to_keep]
        u = self.u[self.cols_to_keep]
        return A,b,sense,c,l,u
    
    def reduce(self):
        '''
        简化过程主流程
        '''
        ## 中间变量初始化
        self.x_preprocess = self.l.copy()
        self.x_preprocess[self.l <= -INF] = 0
        self.A_preprocess = []
        self.rows_to_keep = np.ones((self.n,),dtype=bool)
        self.cols_to_keep = np.ones((self.m,),dtype=bool)
        
        ## 循环地组合使用多种简化手段
        tt = time.time()
        curr_size = (np.sum(self.rows_to_keep),np.sum(self.cols_to_keep))
        count_outer,if_diff_outer = 0,True
        while if_diff_outer and count_outer < 10 and (time.time() - tt) < 120:
            count_inner,if_diff_inner = 0,True
            while if_diff_inner and count_inner < 10 and (time.time() - tt) < 120:
                self.check_empty_row() ## 空行
                self.check_empty_col() ## 空列
                self.check_fixed_col() ## 固定值变量
                self.check_obj_col() ## 最优隐含变量
                self.check_singleton_row() ## 单变量约束

                prev_size = curr_size
                curr_size = (np.sum(self.rows_to_keep),np.sum(self.cols_to_keep))
                if_diff_inner = (prev_size != curr_size) ## 检查是否有简化进展
                count_inner += 1

            self.check_implied_row() ## 可行隐含约束
            self.check_doubleton_row() ## 双变量约束
            prev_size = curr_size
            curr_size = (np.sum(self.rows_to_keep),np.sum(self.cols_to_keep))
            if_diff_outer = (prev_size != curr_size) ## 检查是否有简化进展
            count_outer += 1

        ## 完成主要简化逻辑后，再对输入项进行shape上的化简
        A,b,sense,c,l,u = self.drop_row_col()
        return A,b,sense,c,l,u
    
    def restore(self,sol):
        '''
        根据简化过程中的中间结果，对变量进行恢复；这里只考虑对原始变量x的后处理
        '''
        self.x_preprocess[self.cols_to_keep] = sol.x
        for item in self.A_preprocess[::-1]:
            col_rm,col_keep,ai_col_rm,ai_col_keep,bi = item
            self.x_preprocess[col_rm] = (bi - ai_col_keep * self.x_preprocess[col_keep]) / ai_col_rm
        sol.x = self.x_preprocess
        return sol

### 整理

接下来，通过类Preprocessor对上述预处理方法进行封装，然后将相关预处理/后处理方法加入到DualSimplexSolver.solve中。

In [6]:
class Preprocessor(object):
    def __init__(self):
        self.reducer = None
        self.w_row = None
        self.w_col = None
    
    def preprocess(self,A,b,sense,c,l,u):
        '''
        预处理流程
        '''
        ## 首先对问题进行简化
        print('before reduction: ',A.shape,A.nnz)
        tt = time.time()
        self.reducer = Reducer(A,b,sense,c,l,u)
        A,b,sense,c,l,u = self.reducer.reduce()
        print('after reduction: ',A.shape,A.nnz)
        print('reduction process time: {:.2f}'.format(time.time() - tt))

        ## 然后通过系数缩放降低数值难度
        A,b,c,l,u,(w_row,w_col) = scaling(A,b,c,l,u)
        self.w_row,self.w_col = w_row,w_col

        ## 最后进行问题转化
        A,b,c,l,u = transform(A,b,sense,c,l,u)
        return A,b,c,l,u

    def postprocess(self,sol):
        '''
        后处理流程
        '''
        ## 首先去除形式转化后加入的逻辑变量
        n = len(sol.lam)
        m = len(sol.x) - n
        sol.x,sol.s = sol.x[:m],sol.s[:m]
        if self.w_row is not None and self.w_col is not None:
            ## 然后根据权重对各变量取值进行调整
            sol = descaling(sol,self.w_row,self.w_col)
            
        sol.x[np.isnan(sol.x)] = 0
        if self.reducer is not None:
            ## 最后对简化的变量进行恢复
            self.reducer.restore(sol)
        return sol

In [7]:
class DualSimplexSolverPlus(DualSimplexSolver):
    '''
    继承DualSimplexSolver来沿用已有的DS算法实现；
    修改solve入口来加入预处理
    '''
    
    def solve(self,A_raw,b_raw,sense_raw,c_raw,l_raw,u_raw):
        '''
        主求解入口
        '''
        self.global_info = {'count':0,'start_time':time.time(),'phase':1,'fallback_stack':[],'c_raw':None}
        
        ## 预处理
        self.preprocessor = Preprocessor()
        A,b,sense,c,l,u = A_raw.copy(),b_raw.copy(),sense_raw.copy(),c_raw.copy(),l_raw.copy(),u_raw.copy()
        A,b,c,l,u = self.preprocessor.preprocess(A,b,sense,c,l,u)

        ## 初始化
        problem = Problem(A,b,c,l,u)
        self.global_info['c_raw'] = problem.c.copy()
        # problem = self._perturb(problem)
        n,m = A.shape
        idxB = np.arange(m-n,m,1,dtype=int)
        basis = Basis(A)
        basis.init_DSE_weights()
        basis.reset_basis_idx(idxB)
        problem,sol,basis = self._refactorize(problem,sol=None,basis=basis)
        self.global_info['rollback_stack'] = [(0,1,basis.copy(),sol.copy())]

        ## 开始DS迭代流程
        status,problem,sol,basis = self._solve(problem,sol,basis)
        
        ## 求解完成后做后处理
        sol = self.preprocessor.postprocess(sol)

        return status,sol,basis

### 效果检验

下面通过测试来看一下预处理流程的效果。为了让差别更明显，这里筛选了规模较大的KEN系列问题。随机种子仍然固定为32。

首先调用没有预处理的求解流程。

In [8]:
ds = DualSimplexSolver()
fnames = [fname for fname in sorted(glob.glob('netlib/*.SIF')) if 'KEN' in fname]
test(ds.solve,max_problem_size=1e5,model_fnames=fnames,random_seed=32)

random seed = 32.

Problem name: KEN-07, size: ((2426, 3602),8404).
[----Launch PuLP----]
[----Launch Solver----]
3011 (P2)  (OPT) Obj Primal -6.7952e+08 Dual -6.7952e+08
[----Begin evaluation----]
PuLP Eval: con inf=0.0000e+00,var inf=0.0000e+00,obj=-6.7952e+08. Status: 1.
Solver Eval: con inf=3.8725e-12,var inf=1.9540e-13,obj=-6.7952e+08. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.210, Solver = 3.343.
Two solvers match: PuLP = -6.7952e+08, Solver = -6.7952e+08.

Problem name: KEN-11, size: ((14694, 21349),49058).
[----Launch PuLP----]
[----Launch Solver----]
10000 (P2)  Obj Primal -7.7153e+09 Dual -7.7153e+09  Primal Inf 1.5367e+05 (4799)
17552 (P2)  (OPT) Obj Primal -6.9724e+09 Dual -6.9724e+09
[----Begin evaluation----]
PuLP Eval: con inf=0.0000e+00,var inf=0.0000e+00,obj=-6.9724e+09. Status: 1.
Solver Eval: con inf=1.1051e-10,var inf=1.9185e-11,obj=-6.9724e+09. Status: SolveStatus.OPT.
Elapsed time: PuLP = 1.447, Solver = 81.998.
Two solvers match: PuLP = -6.9724e+09, Solver 

然后调用加入了预处理的求解流程。

In [9]:
ds_p = DualSimplexSolverPlus()
fnames = [fname for fname in sorted(glob.glob('netlib/*.SIF')) if 'KEN' in fname]
test(ds_p.solve,max_problem_size=1e5,model_fnames=fnames,random_seed=32)

random seed = 32.

Problem name: KEN-07, size: ((2426, 3602),8404).
[----Launch PuLP----]
[----Launch Solver----]
before reduction:  (2426, 3602) 8404
after reduction:  (947, 2123) 5014
reduction process time: 0.07
1160 (P2)  (OPT) Obj Primal -5.7440e+08 Dual -5.7440e+08
[----Begin evaluation----]
PuLP Eval: con inf=0.0000e+00,var inf=0.0000e+00,obj=-6.7952e+08. Status: 1.
Solver Eval: con inf=4.7093e-10,var inf=1.1181e-11,obj=-6.7952e+08. Status: SolveStatus.OPT.
Elapsed time: PuLP = 0.230, Solver = 0.982.
Two solvers match: PuLP = -6.7952e+08, Solver = -6.7952e+08.

Problem name: KEN-11, size: ((14694, 21349),49058).
[----Launch PuLP----]
[----Launch Solver----]
before reduction:  (14694, 21349) 49058
after reduction:  (5729, 12384) 29808
reduction process time: 1.44
7286 (P2)  (OPT) Obj Primal -6.1710e+09 Dual -6.1710e+09
[----Begin evaluation----]
PuLP Eval: con inf=0.0000e+00,var inf=0.0000e+00,obj=-6.9724e+09. Status: 1.
Solver Eval: con inf=3.2306e-08,var inf=2.8607e-09,obj=-6.9

根据上面的求解结果可知，（至少对于KEN系列问题，）预处理流程可以显著降低问题的变量和约束数量，进而使得所需求解时间大幅缩短。